# Импорты

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Подгрузка данных

In [2]:
df = pd.read_csv('marketing_AB.csv')
df.head()

,Unnamed: 0,user id,test group,converted,total ads,most ads day,most ads hour
0,0,1069124,ad,False,130,Monday,20
1,1,1119715,ad,False,93,Tuesday,22
2,2,1144181,ad,False,21,Tuesday,18
3,3,1435133,ad,False,355,Tuesday,10
4,4,1015700,ad,False,276,Friday,14


## Посмотр типа данных и содержание каждого столбца

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588101 entries, 0 to 588100
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Unnamed: 0     588101 non-null  int64 
 1   user id        588101 non-null  int64 
 2   test group     588101 non-null  object
 3   converted      588101 non-null  bool  
 4   total ads      588101 non-null  int64 
 5   most ads day   588101 non-null  object
 6   most ads hour  588101 non-null  int64 
dtypes: bool(1), int64(4), object(2)
memory usage: 27.5+ MB


## Проверка на null

In [4]:
df.isnull().sum()

Unnamed: 0       0
user id          0
test group       0
converted        0
total ads        0
most ads day     0
most ads hour    0
dtype: int64

## Приведение к типу данных

In [5]:
df['test group'] = df['test group'].astype('category')
df['most ads day'] = df['most ads day'].astype('category')

## Проверка дубпликатов

In [6]:
duplicates = df[df.duplicated(subset='user id', keep= False)]
if not duplicates.empty:
    df = df[~df['user id'].isin(duplicates['user id'])]

duplicates

,Unnamed: 0,user id,test group,converted,total ads,most ads day,most ads hour


 Пользователей, оказавшихся в обеих группах нет.

# Первичный анализ результатов A/B-тестирования

## Вспомогательные показатели для контрольной и тестовой групп (количество посещений сайта, суммарное количество совершённых покупок)

In [7]:
summary = df.groupby('test group').agg(visits=('user id', 'count'), total_conversions=('converted', 'sum')).reset_index()
print(summary)

  test group  visits  total_conversions
0         ad  564577              14423
1        psa   23524                420


C:\Users\vvser\AppData\Local\Temp\ipykernel_68348\1824790529.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary = df.groupby('test group').agg(visits=('user id', 'count'), total_conversions=('converted', 'sum')).reset_index()


### Вывод

1. Количество посещений группы Ad значительно превышает количество посещений группы PSA. 
    Это может указывать на неравномерность в трафике, что может повлиять на результаты A/B-тестирования и, следовательно, на выводы о конверсии и эффективности каждой страницы.
    Классы несбалансированы.
2. Группа Ad имеет существенно большее количество посещений, почти в 24 раза больше, чем у группы PSA. Это может привести к искажению результатов, так как вероятное влияние трафика может исказить выводы.






##  Ключевые показатели

### Конверсия

In [8]:
summary['conversion_rate'] = summary['total_conversions'] / summary['visits']
summary['conversion_rate']

0    0.025547
1    0.017854
Name: conversion_rate, dtype: float64

### Cреднее количество рекламы, увиденной пользователем

In [9]:
summary['everage_add'] = df.groupby('test group')['total ads'].mean().values
summary['everage_add'] 

C:\Users\vvser\AppData\Local\Temp\ipykernel_68348\1820980334.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary['everage_add'] = df.groupby('test group')['total ads'].mean().values


0    24.823365
1    24.761138
Name: everage_add, dtype: float64

### Выводы

In [10]:
print(summary)

  test group  visits  total_conversions  conversion_rate  everage_add
0         ad  564577              14423         0.025547    24.823365
1        psa   23524                420         0.017854    24.761138


1. Вариант A(AD) значительно эффективнее по показателю конверсии по сравнению с вариантом B(PSA). 
    Выше уровень конверсии указывает на то, что в группе A больше пользователей, совершивших желаемое действие (например, покупка или подписка), по сравнению с группой B.
2. В обеих группах среднее количество увиденной рекламы (average_add) примерно одинаковое: 
    у группы A — 24.82, у группы B — 24.76. Этот показатель практически не отличается и, следовательно, не может служить объяснением различий в конверсии между группами.
3. Группа A (ad) получила 564577 посещений, тогда как группа B (psa) — всего 23524 посещения. 
    Это значительное различие в количестве посещений может указывать на то, что группа A испытала более широкий охват, что также может повлиять на результаты конверсии.

# Cтатистический анализ результатов A/B-тестирования

## Статистические гипотезы: 

### 1. Вопрос: Будет ли кампания успешной?

* Нулевая гипотеза (H0): Кампания не успешна (уровень конверсии не превышает определенного порога, например, средний уровень конверсии для предыдущих кампаний).
* Альтернативная гипотеза (H1): Кампания успешна (уровень конверсии превышает установленный порог).

Выбор статистического теста:

Для проверки этой гипотезы можно использовать тест пропорций (например, Z-тест для пропорций), чтобы сравнить уровень конверсии текущей кампании с заданным порогом. 

### 2. Вопрос: Если кампания была успешной, насколько этот успех можно объяснить рекламой?

* Нулевая гипотеза (H0): Уровень конверсии не зависит от влияния рекламы (различия в уровнях конверсии между группами, например, между целевой (рекламой) и контрольной группами равны).
* Альтернативная гипотеза (H1): Уровень конверсии зависит от влияния рекламы (различия в уровнях конверсии между группами значимы).

Выбор статистического теста:

Для проверки этой гипотезы можно использовать T-тест для независимых выборок (если данные нормально распределены и дисперсии равны) для сравнения уровней конверсии двух групп (рекламная и контрольная).

## Z-ТЕСТ ДЛЯ ПРОПОРЦИЙ

In [11]:
from statsmodels.stats.proportion import proportions_ztest

In [12]:
count = summary['total_conversions'].values
nops = summary['visits'].values

stat, p_val = proportions_ztest(count, nops)
print(f'Z test statistics {stat} p_val {p_val}')

Z test statistics 7.3700781265454145 p_val 1.7052807161559727e-13


Z-статистика: Значение Z-статистики 7.37 указывает на то, что разница между конверсиями в двух группах (например, тестовой и контрольной) значительна. Это значение значительно превышает стандартные критические значения (обычно ±1.96 для уровня значимости 5%), что говорит о том, что наблюдаемая разница не является случайной. p-значение: Очень низкое p-значение (1.70e-13) говорит о том, что вероятность того, что такая разница могла бы возникнуть случайно при условии, что нулевая гипотеза верна (то есть конверсии в обеих группах одинаковы), практически равна нулю. Обычно, если p-значение меньше 0.05, мы отвергаем нулевую гипотезу. Таким образом, можно сделать вывод, что: Статистически значимая разница: Конверсии в группах различаются статистически значимо. Выводы для бизнеса: Если одна из групп имеет значительно более высокую конверсию, это может свидетельствовать о том, что изменения (например, реклама или дизайн) положительно сказались на эффективности кампании. Эти результаты позволяют утверждать, что реклама или изменения в тестируемой группе действительно оказали влияние на конверсии.

## Определение статистических разниц с помщью аппарата статистических тестов

In [13]:
from scipy import stats

group_a =  df[df['test group'] ==  'ad']['total ads']
group_b =  df[df['test group'] ==  'psa']['total ads']

t_stat, p_val = stats.ttest_ind(group_a,group_b)
print(f'ttset statistics {t_stat}, p-val {p_val}')

ttset statistics 0.21391499762385457, p-val 0.8306134186810267


На основании полученных результатов мы не можем отвергнуть нулевую гипотезу (H0). Это значит, что данные не предоставляют статистически значимых доказательств того, что реклама влияет на уровень конверсии.
Вероятнее всего, рекламные усилия не приводят к заметному увеличению конверсии по сравнению с контрольной группой. Возможно, стоит рассмотреть другие маркетинговые стратегии или проводить дальнейшие исследования с более масштабными данными или альтернативными методами анализа.

## Результаты статистических тестов с 95 % доверительные интевалами

In [14]:
import statsmodels.api as sm

conversion_a = summary.loc[summary['test group'] ==  'ad', 'conversion_rate'].values[0]
conversion_b = summary.loc[summary['test group'] ==  'psa', 'conversion_rate'].values[0]

conf_int_a = sm.stats.proportion_confint(count[0], nops[0], alpha=0.05)
conf_int_b = sm.stats.proportion_confint(count[1], nops[1], alpha=0.05)

print(f'95% for group A {conf_int_a}, 95% for group B {conf_int_b}')

95% for group A (0.0251349995427061, 0.025958119730661394), 95% for group B (0.016161914715211324, 0.019546298173753137)


Группа A демонстрирует значительно более высокие конверсии по сравнению с группой B, и результаты тестирования показывают, что различия между группами являются статистически значимыми. Это может иметь важные практические последствия для принятия решений о распределении ресурсов на рекламу.